In [2]:
source('utilities.R')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Package 'mclust' version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.


Attaching package: ‘purrr’


The following object is masked from ‘package:mclust’:

    map


Loading required package: future



# All

In [4]:
df <- read.csv('../Data/palo_alto.csv')
step=15
begin = '2019-01-07' 
end = '2019-09-29'
tz = "US/Pacific"

all_trans <- transactions(df = df,train_starts = begin,train_ends = end,tz = tz)
profile <- load_profile(all_trans,step,tz=tz)
head(profile)

Start,Power
<dttm>,<dbl>
2019-01-07 00:00:00,0
2019-01-07 00:15:00,0
2019-01-07 00:30:00,0
2019-01-07 00:45:00,0
2019-01-07 01:00:00,0
2019-01-07 01:15:00,0


# Predictions

In [1]:
data <- readRDS('../Outputs/transactions.rds')
step=15

In [9]:
library(lubridate)
library(dplyr)
library(furrr)
library(purrr)

In [13]:
time_start <- Sys.time()
plan(multisession)

profiles <- flatten(data) %>%
  future_map_dfr(load_profile, step=step)

plan(sequential)

Sys.time() - time_start

Time difference of 2.275244 mins

In [14]:
head(profiles)

Start,Power
<dttm>,<dbl>
2019-09-30 00:00:00,0.000000
2019-09-30 00:15:00,0.000000
2019-09-30 00:30:00,0.000000
2019-09-30 00:45:00,0.000000
2019-09-30 01:00:00,0.000000
2019-09-30 01:15:00,6.018143


In [15]:
final <- profiles %>%
group_by(Start) %>%
summarise(
  mean = mean(Power),
  q10 = quantile(Power, 0.10),
  q20 = quantile(Power, 0.20),
  q30 = quantile(Power, 0.30),
  q40 = quantile(Power, 0.40),
  q50 = quantile(Power, 0.50),
  q60 = quantile(Power, 0.60),
  q70 = quantile(Power, 0.70),
  q80 = quantile(Power, 0.80),
  q90 = quantile(Power, 0.90)
)

In [18]:
tail(final)

Start,mean,q10,q20,q30,q40,q50,q60,q70,q80,q90
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-10-13 22:30:00,20.31951,5.985223,9.667107,12.708702,16.120175,18.61419,22.01795,25.57902,29.97117,36.09538
2019-10-13 22:45:00,18.25301,5.874742,9.038842,11.879114,14.838233,17.57497,20.25853,23.56801,27.19609,33.17461
2019-10-13 23:00:00,16.50624,4.101919,6.270959,9.993564,12.132435,15.29293,17.98717,21.19104,24.48161,30.26908
2019-10-13 23:15:00,15.06646,3.245373,6.019861,9.172507,11.877530,13.41661,16.60125,19.00111,23.60110,28.31902
2019-10-13 23:30:00,13.82129,3.056548,5.956424,8.156863,10.525283,12.08263,15.19198,17.97711,21.38556,26.92477
2019-10-13 23:45:00,12.47386,0.000000,5.815037,6.063765,9.182706,11.87255,12.74057,16.26054,19.35502,24.28307
